This notebook adds latitude and longitude information to the dataframe created in the previous notebook. The code to generate the postal code, borough, and neighborhood dataframe is in the first cell, with empty columns included for the coordinates.

In [1]:
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")

postal_table=soup.find('table', class_='wikitable sortable')

postalcode=[]
borough=[]
neighborhood=[]

for row in postal_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        
        if cells[1].find(text=True).strip('\n') != 'Not assigned':
            postalcode.append(cells[0].find(text=True).strip('\n'))
            borough.append(cells[1].find(text=True).strip('\n'))

            if cells[2].find(text=True).strip('\n') == 'Not assigned':
                neighborhood.append(cells[1].find(text=True).strip('\n'))
            else:
                neighborhood.append(cells[2].find(text=True).strip('\n'))
                
df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'])
df['PostalCode'] = postalcode
df['Borough'] = borough
df['Neighborhood'] = neighborhood

The coordinates are obtained from the provided Geospatial_Coordinates.csv and added by matching the postal codes with the ones in df.

In [2]:
geo = pd.read_csv('Geospatial_Coordinates.csv')

In [3]:
for i in range(df.shape[0]):
    for j in range(geo.shape[0]):
        if df.iloc[i][0] == geo.iloc[j][0]:
            df.iloc[i].loc['Latitude'] = geo.iloc[j].loc['Latitude']
            df.iloc[i].loc['Longitude'] = geo.iloc[j].loc['Longitude']
        else:
            continue

In [4]:
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6543,-79.3606
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7185,-79.4648
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6623,-79.3895
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6537,-79.5069
99,M4Y,Downtown Toronto,Church and Wellesley,43.6659,-79.3832
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.6627,-79.3216
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6363,-79.4985
